# 变量设置

In [1]:
LLM_PATH = "/data1/LLM/Qwen2-7B-Instruct"
npu_nums = 1

In [2]:
if npu_nums == 2:
    import os
    os.environ.setdefault('CUDA_VISIBLE_DEVICES', '0,1')
else:
    pass

# 模型加载

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    LLM_PATH,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(LLM_PATH)

messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

# prompt = "Give me a short introduction to large language model."
def chat(prompt):
    messages.append({"role": "user", "content": prompt})

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    messages.append({"role": "assistant", "content": response})
    return response

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# chat

In [4]:
chat('你好')

'你好！很高兴能为你提供帮助。有什么我可以为你做的吗？'

In [5]:
chat('请简单介绍下自己')

'我是一个基于AI技术的助手，能够理解和生成自然语言文本。我的功能包括但不限于回答问题、提供建议、生成代码、聊天等。我由阿里云开发和维护，致力于为用户提供高效、智能的服务。无论你需要解答知识性问题、获取信息、学习新技能，还是只是想找个伴聊聊天，我都在这里，随时准备为你服务。'

In [7]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': '你好'},
 {'role': 'assistant', 'content': '你好！很高兴能为你提供帮助。有什么我可以为你做的吗？'},
 {'role': 'user', 'content': '请简单介绍下自己'},
 {'role': 'assistant',
  'content': '我是一个基于AI技术的助手，能够理解和生成自然语言文本。我的功能包括但不限于回答问题、提供建议、生成代码、聊天等。我由阿里云开发和维护，致力于为用户提供高效、智能的服务。无论你需要解答知识性问题、获取信息、学习新技能，还是只是想找个伴聊聊天，我都在这里，随时准备为你服务。'}]